# 🛍️ Generador de Datos Ficticios - Proyecto Demo Ventas Retail

**Autor:** Ana Morales  
**Proyecto:** Sistema de Predicción de Churn + Clustering  
**Descripción:** Genera 2 años de datos de ventas ficticios realistas para demostración de MLOps

---

## 📊 Datos que generará:
- **~5,000 clientes** con diferentes perfiles (VIP, activos, ocasionales, churned)
- **~100,000 transacciones** distribuidas en 24 meses
- **~5,000 devoluciones** (comportamiento realista)
- **Estacionalidad** (picos en Black Friday, Navidad, rebajas)
- **Comportamiento de churn** realista

---

## 🎯 Salida:
3 archivos CSV subidos a `gs://proyecto-demo-ventas/bronze/`:
1. `clientes_raw.csv`
2. `transacciones_raw.csv`
3. `devoluciones_raw.csv`

## 🔧 PASO 1: Instalación de librerías

In [1]:
# Instalar librerías necesarias
!pip install google-cloud-storage faker -q

print("✅ Librerías instaladas correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.8 MB/s eta 0:00:00
✅ Librerías instaladas correctamente


## 🔐 PASO 2: Autenticación con Google Cloud

**IMPORTANTE:** Ejecuta esta celda y sigue las instrucciones en pantalla para autenticarte.

In [3]:
from google.colab import auth
auth.authenticate_user()

print("✅ Autenticación completada")

✅ Autenticación completada


## 📦 PASO 3: Imports y configuración

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
import random
from google.cloud import storage
import io

# Configuración
fake = Faker('es_ES')  # Datos en español
Faker.seed(42)  # Reproducibilidad
np.random.seed(42)
random.seed(42)

# Parámetros del bucket
BUCKET_NAME = 'proyecto-demo-ventas'
PROJECT_ID = 'balmy-parser-461209-c5'

print("✅ Configuración cargada")
print(f"📦 Bucket destino: gs://{BUCKET_NAME}/bronze/")

✅ Configuración cargada
📦 Bucket destino: gs://proyecto-demo-ventas/bronze/


## 👥 PASO 4: Generar Clientes Ficticios

In [5]:
def generar_clientes(num_clientes=5000):
    """
    Genera clientes ficticios con diferentes perfiles:
    - VIP (10%): Alta frecuencia, alto valor
    - Activos (40%): Compra regular
    - Ocasionales (30%): Compra esporádica
    - Churned (20%): Dejaron de comprar
    """

    clientes = []
    fecha_inicio = datetime(2023, 1, 1)
    fecha_fin = datetime(2024, 6, 30)  # 18 meses de histórico

    for i in range(num_clientes):
        # Fecha primera compra aleatoria en los últimos 18 meses
        dias_aleatorios = random.randint(0, (fecha_fin - fecha_inicio).days)
        fecha_primera_compra = fecha_inicio + timedelta(days=dias_aleatorios)

        # Asignar perfil
        rand = random.random()
        if rand < 0.10:
            perfil = 'VIP'
        elif rand < 0.50:
            perfil = 'Activo'
        elif rand < 0.80:
            perfil = 'Ocasional'
        else:
            perfil = 'Churned'

        cliente = {
            'cliente_id': f'CLI_{i+1:05d}',
            'nombre': fake.name(),
            'email': fake.email(),
            'ciudad': fake.city(),
            'codigo_postal': fake.postcode(),
            'fecha_primera_compra': fecha_primera_compra.strftime('%Y-%m-%d'),
            'perfil': perfil,
            'fecha_creacion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        clientes.append(cliente)

    df_clientes = pd.DataFrame(clientes)

    print(f"✅ Generados {len(df_clientes):,} clientes")
    print(f"\n📊 Distribución de perfiles:")
    print(df_clientes['perfil'].value_counts())

    return df_clientes

# Generar clientes
df_clientes = generar_clientes(5000)
df_clientes.head()

✅ Generados 5,000 clientes

📊 Distribución de perfiles:
perfil
Activo       2041
Ocasional    1496
Churned       976
VIP           487
Name: count, dtype: int64


,cliente_id,nombre,email,ciudad,codigo_postal,fecha_primera_compra,perfil,fecha_creacion
0,CLI_00001,Albano Llopis Hierro,emiliocalatayud@example.net,Palencia,28651,2023-04-25,VIP,2026-01-03 18:27:39
1,CLI_00002,Albino Bueno Gil,anselmaamor@example.net,Teruel,43590,2023-10-09,Activo,2026-01-03 18:27:39
2,CLI_00003,Hortensia de Sanchez,nydia02@example.net,Guipúzcoa,19210,2023-05-23,Ocasional,2026-01-03 18:27:39
3,CLI_00004,Cándido Navarro Calatayud,pintocleto@example.com,Pontevedra,18335,2023-03-31,Ocasional,2026-01-03 18:27:39
4,CLI_00005,Pánfilo Segovia Casanovas,jbermudez@example.com,Ourense,13601,2023-02-02,VIP,2026-01-03 18:27:39


## 🛒 PASO 5: Generar Transacciones Realistas

In [6]:
def generar_transacciones(df_clientes, fecha_inicio='2023-01-01', fecha_fin='2024-12-31'):
    """
    Genera transacciones realistas basadas en el perfil del cliente.
    Incluye estacionalidad y comportamiento de churn.
    """

    # Catálogo de productos tech
    productos = [
        {'nombre': 'Laptop Gaming', 'precio_base': 1200, 'categoria': 'Ordenadores'},
        {'nombre': 'Laptop Ultrabook', 'precio_base': 800, 'categoria': 'Ordenadores'},
        {'nombre': 'PC Sobremesa', 'precio_base': 600, 'categoria': 'Ordenadores'},
        {'nombre': 'Monitor 27"', 'precio_base': 300, 'categoria': 'Periféricos'},
        {'nombre': 'Teclado Mecánico', 'precio_base': 120, 'categoria': 'Periféricos'},
        {'nombre': 'Ratón Gaming', 'precio_base': 60, 'categoria': 'Periféricos'},
        {'nombre': 'Auriculares Bluetooth', 'precio_base': 80, 'categoria': 'Audio'},
        {'nombre': 'Webcam HD', 'precio_base': 50, 'categoria': 'Periféricos'},
        {'nombre': 'SSD 1TB', 'precio_base': 100, 'categoria': 'Componentes'},
        {'nombre': 'RAM 16GB', 'precio_base': 80, 'categoria': 'Componentes'},
        {'nombre': 'Tablet', 'precio_base': 400, 'categoria': 'Tablets'},
        {'nombre': 'Smartphone', 'precio_base': 500, 'categoria': 'Móviles'},
        {'nombre': 'Smartwatch', 'precio_base': 250, 'categoria': 'Wearables'},
        {'nombre': 'Impresora', 'precio_base': 150, 'categoria': 'Periféricos'},
        {'nombre': 'Router WiFi', 'precio_base': 70, 'categoria': 'Redes'},
    ]

    transacciones = []

    fecha_inicio_dt = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    fecha_fin_dt = datetime.strptime(fecha_fin, '%Y-%m-%d')

    # Frecuencia de compra según perfil
    frecuencia_compra = {
        'VIP': (15, 30),        # Cada 15-30 días
        'Activo': (45, 90),     # Cada 1.5-3 meses
        'Ocasional': (120, 180), # Cada 4-6 meses
        'Churned': (180, 365)   # Cada 6-12 meses (y dejan de comprar)
    }

    transaccion_id = 1

    for _, cliente in df_clientes.iterrows():
        fecha_primera = datetime.strptime(cliente['fecha_primera_compra'], '%Y-%m-%d')
        perfil = cliente['perfil']

        # Calcular número de transacciones según perfil
        dias_rango = frecuencia_compra[perfil]
        fecha_actual = fecha_primera

        # Límite de transacciones según perfil
        max_transacciones = {
            'VIP': 30,
            'Activo': 15,
            'Ocasional': 6,
            'Churned': 3
        }

        num_trans = 0

        while fecha_actual <= fecha_fin_dt and num_trans < max_transacciones[perfil]:
            # Estacionalidad: más probabilidad de compra en temporadas clave
            mes = fecha_actual.month
            factor_estacionalidad = 1.0

            # Black Friday (noviembre)
            if mes == 11:
                factor_estacionalidad = 2.0
            # Navidad (diciembre)
            elif mes == 12:
                factor_estacionalidad = 1.8
            # Rebajas (enero, julio)
            elif mes in [1, 7]:
                factor_estacionalidad = 1.5
            # Back to school (septiembre)
            elif mes == 9:
                factor_estacionalidad = 1.3

            # Decidir si compra (más probabilidad en temporada alta)
            if random.random() < (0.7 * factor_estacionalidad):
                # Número de productos por transacción
                num_productos = random.choices(
                    [1, 2, 3, 4],
                    weights=[0.5, 0.3, 0.15, 0.05]
                )[0]

                productos_comprados = random.sample(productos, num_productos)

                importe_total = 0
                descuento_total = 0

                for prod in productos_comprados:
                    precio_base = prod['precio_base']
                    # Variación de precio ±10%
                    precio = precio_base * random.uniform(0.9, 1.1)

                    # Descuentos según temporada
                    descuento_pct = 0
                    if mes == 11:  # Black Friday 20-40%
                        descuento_pct = random.uniform(0.20, 0.40)
                    elif mes in [1, 7]:  # Rebajas 10-30%
                        descuento_pct = random.uniform(0.10, 0.30)
                    elif perfil == 'VIP':  # VIP siempre 5-15%
                        descuento_pct = random.uniform(0.05, 0.15)

                    descuento = precio * descuento_pct
                    precio_final = precio - descuento

                    importe_total += precio_final
                    descuento_total += descuento

                # Crear transacción
                transaccion = {
                    'transaccion_id': f'TXN_{transaccion_id:08d}',
                    'cliente_id': cliente['cliente_id'],
                    'fecha_transaccion': fecha_actual.strftime('%Y-%m-%d'),
                    'hora_transaccion': f"{random.randint(10,21):02d}:{random.randint(0,59):02d}:00",
                    'num_productos': num_productos,
                    'categorias': ','.join(set([p['categoria'] for p in productos_comprados])),
                    'importe_bruto': round(importe_total + descuento_total, 2),
                    'descuento': round(descuento_total, 2),
                    'importe_neto': round(importe_total, 2),
                    'metodo_pago': random.choice(['Tarjeta', 'PayPal', 'Transferencia']),
                    'canal': random.choice(['Web', 'App', 'Tienda física']),
                }

                transacciones.append(transaccion)
                transaccion_id += 1
                num_trans += 1

            # Siguiente fecha de compra
            dias_hasta_siguiente = random.randint(*dias_rango)
            fecha_actual += timedelta(days=dias_hasta_siguiente)

            # Si es Churned, dejar de comprar después de cierto tiempo
            if perfil == 'Churned' and num_trans >= 2:
                # Dejan de comprar hace 6+ meses
                if fecha_actual > datetime(2024, 6, 30):
                    break

    df_transacciones = pd.DataFrame(transacciones)

    print(f"\n✅ Generadas {len(df_transacciones):,} transacciones")
    print(f"📊 Rango de fechas: {df_transacciones['fecha_transaccion'].min()} - {df_transacciones['fecha_transaccion'].max()}")
    print(f"💰 Importe total: €{df_transacciones['importe_neto'].sum():,.2f}")
    print(f"📦 Productos vendidos: {df_transacciones['num_productos'].sum():,}")

    return df_transacciones

# Generar transacciones
df_transacciones = generar_transacciones(df_clientes)
df_transacciones.head(10)


✅ Generadas 26,674 transacciones
📊 Rango de fechas: 2023-01-01 - 2024-12-31
💰 Importe total: €13,446,338.39
📦 Productos vendidos: 46,552


,transaccion_id,cliente_id,fecha_transaccion,hora_transaccion,num_productos,categorias,importe_bruto,descuento,importe_neto,metodo_pago,canal
0,TXN_00000001,CLI_00001,2023-06-04,11:57:00,3,"Ordenadores,Periféricos",1619.47,219.74,1399.73,PayPal,App
1,TXN_00000002,CLI_00001,2023-06-28,20:49:00,1,Redes,67.51,7.28,60.24,Tarjeta,App
2,TXN_00000003,CLI_00001,2023-07-17,20:29:00,1,Periféricos,110.05,23.48,86.58,Transferencia,Web
3,TXN_00000004,CLI_00001,2023-09-10,17:26:00,2,"Periféricos,Móviles",558.67,64.28,494.39,PayPal,App
4,TXN_00000005,CLI_00001,2023-11-07,18:23:00,1,Redes,67.23,16.71,50.52,Tarjeta,Tienda física
5,TXN_00000006,CLI_00001,2023-12-01,20:06:00,3,"Ordenadores,Tablets,Audio",991.06,99.17,891.90,PayPal,Tienda física
6,TXN_00000007,CLI_00001,2023-12-19,21:03:00,2,"Periféricos,Móviles",597.45,39.50,557.94,Transferencia,Web
7,TXN_00000008,CLI_00001,2024-01-07,10:09:00,4,"Ordenadores,Periféricos,Wearables,Tablets",1550.83,291.76,1259.07,Transferencia,Web
8,TXN_00000009,CLI_00001,2024-01-25,10:59:00,1,Ordenadores,1279.85,241.59,1038.26,Tarjeta,App
9,TXN_00000010,CLI_00001,2024-02-21,19:43:00,1,Periféricos,320.50,33.55,286.96,PayPal,Tienda física


## 🔄 PASO 6: Generar Devoluciones

In [7]:
def generar_devoluciones(df_transacciones, tasa_devolucion=0.05):
    """
    Genera devoluciones realistas (5% de transacciones aprox.)
    """

    # Seleccionar transacciones que tendrán devolución
    num_devoluciones = int(len(df_transacciones) * tasa_devolucion)
    transacciones_con_devolucion = df_transacciones.sample(n=num_devoluciones, random_state=42)

    devoluciones = []

    motivos = [
        'Producto defectuoso',
        'No cumple expectativas',
        'Talla/especificaciones incorrectas',
        'Llegó dañado',
        'Cambio de opinión',
        'Incompatibilidad técnica'
    ]

    for idx, txn in transacciones_con_devolucion.iterrows():
        # Fecha de devolución: 3-30 días después de la compra
        fecha_compra = datetime.strptime(txn['fecha_transaccion'], '%Y-%m-%d')
        dias_devolucion = random.randint(3, 30)
        fecha_devolucion = fecha_compra + timedelta(days=dias_devolucion)

        # Devolver productos parcialmente o totalmente
        if random.random() < 0.7:  # 70% devolución total
            importe_devuelto = txn['importe_neto']
            productos_devueltos = txn['num_productos']
        else:  # 30% devolución parcial
            productos_devueltos = random.randint(1, txn['num_productos'])
            importe_devuelto = txn['importe_neto'] * (productos_devueltos / txn['num_productos'])

        devolucion = {
            'devolucion_id': f'DEV_{len(devoluciones)+1:06d}',
            'transaccion_id': txn['transaccion_id'],
            'cliente_id': txn['cliente_id'],
            'fecha_devolucion': fecha_devolucion.strftime('%Y-%m-%d'),
            'productos_devueltos': productos_devueltos,
            'importe_devuelto': round(importe_devuelto, 2),
            'motivo': random.choice(motivos),
            'estado': random.choice(['Procesada', 'Procesada', 'Procesada', 'Pendiente']),
        }

        devoluciones.append(devolucion)

    df_devoluciones = pd.DataFrame(devoluciones)

    print(f"\n✅ Generadas {len(df_devoluciones):,} devoluciones")
    print(f"📊 Tasa de devolución: {len(df_devoluciones)/len(df_transacciones)*100:.2f}%")
    print(f"💸 Importe devuelto total: €{df_devoluciones['importe_devuelto'].sum():,.2f}")

    return df_devoluciones

# Generar devoluciones
df_devoluciones = generar_devoluciones(df_transacciones, tasa_devolucion=0.05)
df_devoluciones.head()


✅ Generadas 1,333 devoluciones
📊 Tasa de devolución: 5.00%
💸 Importe devuelto total: €634,382.19


,devolucion_id,transaccion_id,cliente_id,fecha_devolucion,productos_devueltos,importe_devuelto,motivo,estado
0,DEV_000001,TXN_00002502,CLI_00474,2024-02-18,1,390.66,Producto defectuoso,Procesada
1,DEV_000002,TXN_00018439,CLI_03462,2024-08-08,2,622.95,No cumple expectativas,Procesada
2,DEV_000003,TXN_00005358,CLI_01018,2023-03-01,1,626.88,Talla/especificaciones incorrectas,Procesada
3,DEV_000004,TXN_00020934,CLI_03959,2024-11-12,1,163.73,Llegó dañado,Pendiente
4,DEV_000005,TXN_00008450,CLI_01586,2024-08-29,3,537.89,Producto defectuoso,Procesada


## 📊 PASO 7: Resumen de Datos Generados

In [8]:
print("=" * 60)
print("📊 RESUMEN DE DATOS GENERADOS")
print("=" * 60)
print(f"\n👥 CLIENTES: {len(df_clientes):,}")
print(df_clientes['perfil'].value_counts())

print(f"\n🛒 TRANSACCIONES: {len(df_transacciones):,}")
print(f"   Periodo: {df_transacciones['fecha_transaccion'].min()} → {df_transacciones['fecha_transaccion'].max()}")
print(f"   Importe total: €{df_transacciones['importe_neto'].sum():,.2f}")
print(f"   Ticket promedio: €{df_transacciones['importe_neto'].mean():.2f}")

print(f"\n🔄 DEVOLUCIONES: {len(df_devoluciones):,}")
print(f"   Tasa: {len(df_devoluciones)/len(df_transacciones)*100:.2f}%")
print(f"   Importe devuelto: €{df_devoluciones['importe_devuelto'].sum():,.2f}")

print(f"\n💾 Tamaño estimado de archivos:")
print(f"   clientes_raw.csv: ~{len(df_clientes) * 150 / 1024:.1f} KB")
print(f"   transacciones_raw.csv: ~{len(df_transacciones) * 200 / 1024:.1f} KB")
print(f"   devoluciones_raw.csv: ~{len(df_devoluciones) * 150 / 1024:.1f} KB")

print("\n" + "=" * 60)

📊 RESUMEN DE DATOS GENERADOS

👥 CLIENTES: 5,000
perfil
Activo       2041
Ocasional    1496
Churned       976
VIP           487
Name: count, dtype: int64

🛒 TRANSACCIONES: 26,674
   Periodo: 2023-01-01 → 2024-12-31
   Importe total: €13,446,338.39
   Ticket promedio: €504.10

🔄 DEVOLUCIONES: 1,333
   Tasa: 5.00%
   Importe devuelto: €634,382.19

💾 Tamaño estimado de archivos:
   clientes_raw.csv: ~732.4 KB
   transacciones_raw.csv: ~5209.8 KB
   devoluciones_raw.csv: ~195.3 KB



## ☁️ PASO 8: Subir archivos a Google Cloud Storage

In [9]:
def subir_a_gcs(df, bucket_name, blob_name):
    """
    Sube un DataFrame a Google Cloud Storage como CSV
    """
    try:
        # Crear cliente de Storage
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Convertir DataFrame a CSV en memoria
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False, encoding='utf-8')

        # Subir a GCS
        blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')

        print(f"✅ Subido: gs://{bucket_name}/{blob_name} ({len(df):,} filas)")
        return True

    except Exception as e:
        print(f"❌ Error al subir {blob_name}: {str(e)}")
        return False

# Subir los 3 archivos
print("📤 Subiendo archivos a Google Cloud Storage...\n")

subir_a_gcs(df_clientes, BUCKET_NAME, 'bronze/clientes/clientes_raw.csv')
subir_a_gcs(df_transacciones, BUCKET_NAME, 'bronze/transacciones/transacciones_raw.csv')
subir_a_gcs(df_devoluciones, BUCKET_NAME, 'bronze/devoluciones/devoluciones_raw.csv')

print("\n🎉 ¡Proceso completado!")
print(f"\n📂 Archivos disponibles en: gs://{BUCKET_NAME}/bronze/")
print("\n🔗 Verifica en: https://console.cloud.google.com/storage/browser/{}".format(BUCKET_NAME))

📤 Subiendo archivos a Google Cloud Storage...

✅ Subido: gs://proyecto-demo-ventas/bronze/clientes/clientes_raw.csv (5,000 filas)
✅ Subido: gs://proyecto-demo-ventas/bronze/transacciones/transacciones_raw.csv (26,674 filas)
✅ Subido: gs://proyecto-demo-ventas/bronze/devoluciones/devoluciones_raw.csv (1,333 filas)

🎉 ¡Proceso completado!

📂 Archivos disponibles en: gs://proyecto-demo-ventas/bronze/

🔗 Verifica en: https://console.cloud.google.com/storage/browser/proyecto-demo-ventas


## ✅ VERIFICACIÓN: Listar archivos en el bucket

In [10]:
# Listar archivos en el bucket para verificar
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blobs = list(bucket.list_blobs(prefix='bronze/'))

print(f"📂 Archivos en gs://{BUCKET_NAME}/bronze/:\n")
for blob in blobs:
    size_mb = blob.size / (1024 * 1024)
    print(f"   📄 {blob.name} ({size_mb:.2f} MB)")

print(f"\n✅ Total: {len(blobs)} archivos")

📂 Archivos en gs://proyecto-demo-ventas/bronze/:

   📄 bronze/clientes/clientes_raw.csv (0.52 MB)
   📄 bronze/devoluciones/devoluciones_raw.csv (0.11 MB)
   📄 bronze/transacciones/transacciones_raw.csv (2.53 MB)

✅ Total: 3 archivos


---

## 🎯 PRÓXIMOS PASOS

Ahora que tienes los datos en Cloud Storage, el siguiente paso es:

1. ✅ **Crear dataset en BigQuery**
2. ✅ **Cargar datos desde GCS a BigQuery (Bronze layer)**
3. ✅ **Crear pipelines de transformación (Silver/Gold)**
4. ✅ **Entrenar modelo de churn**
5. ✅ **Crear clustering de clientes**
6. ✅ **Dashboards en Looker Studio**

---

**📧 Creado por:** Ana Morales  
**📅 Fecha:** Enero 2025  
**🎯 Proyecto:** Demo MLOps - Sistema de Predicción de Churn + Clustering